**3 read count files**
- non-normalized read counts
- ranknorm normalized using RNOMni
- log2 normalized from Congyi, this is the one used for the original screen analysis

- S8 vs. S6:
    These cells became non-neurons, the **enrichment** of guides found from S8 / S6 means that these guides discrupt TFs that may be crucial to neuron development. Because these cells failed to become neurons, enrichment of these guides mean TFs responsible for neuron development are disrupted.
    
- S7 vs. S6:
    These cells became neurons. The enrichment of guides found from S7 - S6 means that these guides disrupt TFs that may be repressors of neuron development, or that these targetted TFs are reponsible for maintaining stem cell status. The **depletion** of these guides means that the targeted TFs may be crucial to neuron development (since these cells have become neurons, you wouldn't find many cells with neuron related TFs being disrupted, thus you observe depletion of gRNAs). 
    
- S8 vs. S7:
    S8 are non-neurons and S7 are neurons. The **enrichment** of guides found in neurons, while depleted in non-neurons, mean that these guides disrupte the function of TFs that are responsible for neuron development.

In [670]:
suppressMessages(library(data.table))
suppressMessages(library(tidyverse))

## prepare count tables

In [671]:
#cnts.file = "../../resources/crispr/counts/RC.csv"
cnts.file = "../../resources/crispr/counts/wrangled_raw_counts.tsv" # raw file
cnts.rn.file = "../../resources/crispr/counts/wrangled_ranknorm_counts.tsv" # rank normalized
cnts.l2.file = "/gpfs/commons/groups/sanjana_lab/cdai/TFscreen/RCmNorRatioLog2.lu.csv"

In [672]:
cnts = fread(cnts.file) %>% .[, .(ID=sgRNA, Gene, S6 = Day14_iN, S7=Day14_Tdpos, S8=Day14_Tdneg)]
cnts.l2 = fread(cnts.l2.file) %>% .[, .(ID, Gene, S6, S7, S8)]

In [673]:
# group 10 NTC guides into 1 NTC group
NTC = paste("NonTargeting_Human_", sort(rep(1:100, 10)), sep="")
cnts[str_detect(toupper(Gene), "NONTARGET"), Gene := NTC]

In [674]:
# remove any guides where S6 or iN is 1 or less
# remove genes that have less than 5 guides left at iN or S6
cnts = cnts[S6 > 1]
cnts = cnts[, .(ID, S6, S7, S8, .N), by = Gene][N>4, -c("N")]

In [675]:
selcols = c("S6", "S7", "S8")
# log2 RPM normalize counts
cnts.norm = cnts[, c(
    list(ID = ID, Gene = Gene),
    lapply(.SD, function(x) log2(if_else(x==0, .1, .1*x)*1e6/sum(x)) )
), .SDcols = selcols]

In [676]:
cnts[1:5]
dim(cnts)
cnts.norm[1:5]
dim(cnts.norm)

Gene,ID,S6,S7,S8
<chr>,<chr>,<int>,<int>,<int>
AATF,GUIDES_sg00000,46,23,0
AATF,GUIDES_sg00003,122,35,0
AATF,GUIDES_sg00004,5,0,0
AATF,GUIDES_sg00006,50,34,0
AATF,GUIDES_sg00007,22,16,0


[1] 17899     5

ID,Gene,S6,S7,S8
<chr>,<chr>,<dbl>,<dbl>,<dbl>
GUIDES_sg00000,AATF,1.4236802,0.8385772,-4.135369
GUIDES_sg00003,AATF,2.8308556,1.4442983,-4.135369
GUIDES_sg00004,AATF,-1.7779537,-3.6849848,-4.135369
GUIDES_sg00006,AATF,1.5439744,1.4024781,-4.135369
GUIDES_sg00007,AATF,0.3595499,0.3150152,-4.135369


[1] 17899     5

## functions for computation

In [679]:
makeDT = function(dt, dt.norm) {
    # dt: is raw counts,
    # dt.norm: is the same, but normalized
    # columns: ID, Gene, denom, num
    
    # both denom and num conditions must have at least 2 guides with reads > 0
    genes1 = dt[denom > 0, .N, by = Gene][N>1, Gene] %>% unique
    genes2 = dt[num > 0, .N, by = Gene][N>1, Gene] %>% unique
    genes = intersect(genes1, genes2)
    
    dt = dt[Gene %in% genes & denom > 0 & num > 0]
    dt.norm = dt.norm[Gene %in% genes]
    
    dt.out = dt.norm[dt[, .(ID, Gene)], on = c("Gene", "ID"), nomatch=NULL]
    
    return(dt.out)
}

weightedSum = function(x) {
    # x must be a numerical vector of exactly 2 elements
    # x must be sorted, x[1] <= x[2]
    return(0.25*x[1] + 0.75*x[2]) # more weights no the second rank
}

RRA = function(dt, enrichment) {
    # dt is a dataframe with column ID, Gene, denom, num (which are Log2 RPM normalized reads)
    dt = dt[, .(ID, Gene, denom, num, l2fc = num - denom)] # compute l2fc
    if (enrichment) {
        dt = dt[, .(ID, Gene,l2fc, rk=rank(-l2fc, ties.method = "max"))] # look for enriched guides
    } else {
        dt = dt[, .(ID, Gene,l2fc, rk=rank(l2fc, ties.method = "max"))] # look for depleted guides
    }
    dt = dt[order(Gene, rk), .SD[1:2], by = Gene # select top 2 guides for each gene
            ][, .(rankscore = weightedSum(rk)), by = Gene
             ][order(rankscore)]
    return(dt)
}

## compute enrichment of S8 vs S6

In [680]:
# enrichment of guides in td- vs. iN (S8 VS. S6)
dtS86 = makeDT(dt = cnts[, .(ID, Gene, denom = S6, num = S8)],
               dt.norm = cnts.norm[, .(ID, Gene, denom = S6, num = S8)])
rraS86 = RRA(dtS86, enrichment = T)

## compute enrichment of S8 vs S7

In [681]:
# enrichment of guides in td- vs. td+ (S8 VS. S7)
dtS87 = makeDT(dt = cnts[, .(ID, Gene, denom = S7, num = S8)],
               dt.norm = cnts.norm[, .(ID, Gene, denom = S7, num = S8)])
rraS87 = RRA(dtS87, enrichment = T)

## compute depletion of S7 vs S6

In [682]:
# depletion of guides in td+ vs. iN (S7 VS. S6)
dtS76 = makeDT(dt = cnts[, .(ID, Gene, denom = S6, num = S7)],
               dt.norm = cnts.norm[, .(ID, Gene, denom = S6, num = S7)])
rraS76 = RRA(dtS76, enrichment = F)

## Monte Carlo simulation - S86 enrichment

In [686]:
N_sim = 20 # number of simulations

sim.rraS86.l = list()
for (i in 1:N_sim) {
    sim.cnts = cnts[
        , .(Gene, ID,
           S6 = S6[sample(length(S6))],
           S7 = S7[sample(length(S7))],
           S8 = S8[sample(length(S8))])]
    
    selcols = c("S6", "S7", "S8")
    # log2 RPM normalize counts
    sim.cnts.norm = sim.cnts[, c(list(ID = ID, Gene = Gene),
        lapply(.SD, function(x) log2(if_else(x==0, .1, .1*x)*1e6/sum(x)) )), 
        .SDcols = selcols]
               
    sim.dtS86 = makeDT(dt = sim.cnts[, .(ID, Gene, denom = S6, num = S8)],
               dt.norm = sim.cnts.norm[, .(ID, Gene, denom = S6, num = S8)])
    sim.rraS86 = list(RRA(sim.dtS86, enrichment = T))
    sim.rraS86.l = c(sim.rraS86.l, sim.rraS86)
}

# get ecdf
ecdf.S86 = ecdf(map(sim.rraS86.l, ~.x$rankscore) %>% flatten %>% unlist)

# use ecdf to compute FDR value
rraS86[, FDR := ecdf.S86(rankscore)]

## Monte Carlo simulation - S87 enrichment

In [690]:
N_sim = 20 # number of simulations

sim.rraS87.l = list()
for (i in 1:N_sim) {
    sim.cnts = cnts[
        , .(Gene, ID,
           S6 = S6[sample(length(S6))],
           S7 = S7[sample(length(S7))],
           S8 = S8[sample(length(S8))])]
    
    selcols = c("S6", "S7", "S8")
    # log2 RPM normalize counts
    sim.cnts.norm = sim.cnts[, c(list(ID = ID, Gene = Gene),
        lapply(.SD, function(x) log2(if_else(x==0, .1, .1*x)*1e6/sum(x)) )), 
        .SDcols = selcols]
               
    sim.dtS87 = makeDT(dt = sim.cnts[, .(ID, Gene, denom = S7, num = S8)],
               dt.norm = sim.cnts.norm[, .(ID, Gene, denom = S7, num = S8)])
    sim.rraS87 = list(RRA(sim.dtS87, enrichment = T))
    sim.rraS87.l = c(sim.rraS87.l, sim.rraS87)
}

# get ecdf
ecdf.S87 = ecdf(map(sim.rraS87.l, ~.x$rankscore) %>% flatten %>% unlist)

# use ecdf to compute FDR value
rraS87[, FDR := ecdf.S87(rankscore)]

## Monte Carlo simulation - S76 depletion

In [694]:
N_sim = 20 # number of simulations

sim.rraS76.l = list()
for (i in 1:N_sim) {
    sim.cnts = cnts[
        , .(Gene, ID,
           S6 = S6[sample(length(S6))],
           S7 = S7[sample(length(S7))],
           S8 = S8[sample(length(S8))])]
    
    selcols = c("S6", "S7", "S8")
    # log2 RPM normalize counts
    sim.cnts.norm = sim.cnts[, c(list(ID = ID, Gene = Gene),
        lapply(.SD, function(x) log2(if_else(x==0, .1, .1*x)*1e6/sum(x)) )), 
        .SDcols = selcols]
               
    sim.dtS76 = makeDT(dt = sim.cnts[, .(ID, Gene, denom = S6, num = S7)],
               dt.norm = sim.cnts.norm[, .(ID, Gene, denom = S6, num = S7)])
    sim.rraS76 = list(RRA(sim.dtS76, enrichment = F))
    sim.rraS76.l = c(sim.rraS76.l, sim.rraS76)
}

# get ecdf
ecdf.S76 = ecdf(map(sim.rraS76.l, ~.x$rankscore) %>% flatten %>% unlist)

# use ecdf to compute FDR value
rraS76[, FDR := ecdf.S76(rankscore)]

## Compare with hit list identified previously

In [695]:
hitlist = fread("../../resources/crispr/Hitlist_20191230.csv")

In [696]:
hitlist[1:2]

gene_name,hgnc_symbol
<chr>,<chr>
PHOX2A,PHOX2A
TAF1L,TAF1L


In [702]:
# Generate 3 sets of 200 words
set87 = rraS87[FDR < .1, Gene]
set86 = rraS86[FDR < .1, Gene]
set76 = rraS76[FDR < .1, Gene]

# Helper function to display Venn diagram
display_venn <- function(x, ...){
  grid.newpage()
  venn_object <- venn.diagram(x, filename = NULL, ...)
  grid.draw(venn_object)
}

display_venn(x = list(set86, set87, hitlist$gene_name), 
             category = c("Td- vs iN", "Td- vs Td+", "Hits"),
             alpha = .7, fill = c("tomato1", 'lightblue', 'lightgreen'),
             cat.cex = 1.5, cex = 1.7, label.col = "blue"
            )

In [703]:
rraS87[FDR < .1, Gene]

[1] "ZNF441"                "ZNF440"                "CRY1"                 
 [4] "E2F1"                  "CDX4"                  "BHLHA15"              
 [7] "ZNF311"                "EGR3"                  "ZIC1"                 
[10] "NR1I2"                 "ZNF250"                "ZNF639"               
[13] "PRDM6"                 "DLX5"                  "TOX4"                 
[16] "THRA"                  "BRWD1"                 "CHD5"                 
[19] "CBX4"                  "VAX2"                  "ZBTB18"               
[22] "RAI14"                 "RBPJ"                  "TSHZ3"                
[25] "NKX3-2"                "TAF1L"                 "APBB2"                
[28] "SCRT2"                 "HSFY1"                 "ZNF136"               
[31] "EMX1"                  "UBE2K"                 "CTDP1"                
[34] "JUNB"                  "MECP2"                 "POLR2B"               
[37] "ZZZ3"                  "NonTargeting_Human_22" "ZNF554"               
[40] "KLF10"                 "INSM1"                 "MEOX2"                
[43] "ZNF671"                "NonTargeting_Human_90" "SMAD3"                
[46] "EBF1"                  "ZNF563"                "PAX7"                 
[49] "ZNF141"                "ZFPM1"                 "HIST1H2BC"            
[52] "JUND"                  "HTATIP2"               "PRDM5"                
[55] "GATA1"                 "TFEB"                  "CREBZF"               
[58] "VGLL2"                 "SKI"                   "ZFY"                  
[61] "HIF3A"                 "NonTargeting_Human_13" "ELK3"                 
[64] "NonTargeting_Human_31" "NonTargeting_Human_57" "ZIM3"                 
[67] "ARID1B"

In [704]:
intersect(union(set87, set86), hitlist$gene_name) %>% length
intersect(union(set87, set86), hitlist$gene_name)

[1] 74

[1] "ZNF441"    "ZNF440"    "CRY1"      "E2F1"      "CDX4"      "BHLHA15"  
 [7] "ZNF311"    "EGR3"      "ZIC1"      "NR1I2"     "ZNF250"    "ZNF639"   
[13] "PRDM6"     "DLX5"      "TOX4"      "THRA"      "BRWD1"     "CHD5"     
[19] "CBX4"      "VAX2"      "ZBTB18"    "RAI14"     "RBPJ"      "TSHZ3"    
[25] "NKX3-2"    "TAF1L"     "APBB2"     "SCRT2"     "HSFY1"     "ZNF136"   
[31] "EMX1"      "UBE2K"     "CTDP1"     "JUNB"      "MECP2"     "POLR2B"   
[37] "ZZZ3"      "ZNF554"    "KLF10"     "INSM1"     "MEOX2"     "ZNF671"   
[43] "SMAD3"     "EBF1"      "ZNF563"    "PAX7"      "ZNF141"    "ZFPM1"    
[49] "HIST1H2BC" "JUND"      "HTATIP2"   "PRDM5"     "GATA1"     "TFEB"     
[55] "CREBZF"    "VGLL2"     "SKI"       "ZFY"       "HIF3A"     "ELK3"     
[61] "ZIM3"      "ARID1B"    "ZNF510"    "ISL2"      "TSC22D3"   "ZNF239"   
[67] "PHF12"     "NFKBIB"    "SIX5"      "NEUROG3"   "PAX2"      "LHX4"     
[73] "CTNNB1"    "ZKSCAN3"